# Imports and program variables


In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
import pandas as pd

from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint


from sklearn.model_selection import train_test_split # to split dataset into train and test
from sklearn import preprocessing
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/gabriel/.pyenv/versions/3.6.4/envs/machine-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  

In [2]:
DATASET_PATH = '../src/Features/Original/MFCC_2/'
from time_series_dataset_loader import TimeSeriesDatasetLoader


## Setting up train and test datasets

In [65]:
dataset_loader = TimeSeriesDatasetLoader(DATASET_PATH)
X, y = dataset_loader.get_dataset(type_='russel')

In [66]:
X = np.asarray(X)
y = np.asarray(y)

In [22]:
y

array([ 1, -1,  0, ...,  0, -1,  1])

In [68]:
# Completing with zeros
max_len = len(X[0])
for row in X:
        if len(row) > max_len:
            max_len = len(row)

X = pad_sequences(X, maxlen=max_len, padding='post')

In [18]:
# See X and y details
print(X[:2])
print(X.shape)

print(y[:3])
print(y.shape)

[[[  0   0 -16 ...  10   0 -11]
  [  1   0 -11 ...  13   4  -2]
  [  2   0  -6 ...  11   5   2]
  ...
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]]

 [[  0   0  -2 ...   8   1 -17]
  [  1   0  -1 ...   8   0 -19]
  [  2   0   0 ...   8   2 -20]
  ...
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]]]
(1164, 269, 16)
[ 1 -1  0]
(1164,)


In [69]:
# Split the dataset in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)

print("\nX_train:\n")
print(X_train[0][0])
print(X_train.shape)

print("\nX_test:\n")
print(X_test[0][0])
print(X_test.shape)

print("\ny_train:\n")
print(y_train[:3])
print(y_train.shape)

print("\ny_test:\n")
print(y_test[:3])
print(y_test.shape)


X_train:

[  0   0   6  10  -2 -10   8   3   1   0  -8   1  -2   4   6 -12]
(814, 269, 16)

X_test:

[  0   0  -4  -1  -2  11   8  -6  11   5  10   5   2   7   1 -20]
(350, 269, 16)

y_train:

[3 4 3]
(814,)

y_test:

[3 3 4]
(350,)


In [70]:
# Create categorical matrices
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [71]:
num_rows = X[0].shape[0]
num_columns = X[0].shape[1]
num_channels = 1

X_train = X_train.reshape(X_train.shape[0], num_rows, num_columns, num_channels)
X_test = X_test.reshape(X_test.shape[0], num_rows, num_columns, num_channels)

print("\nX_train:\n")
print(X_train[0][0])
print(X_train.shape)

print("\nX_test:\n")
print(X_test[0][0])
print(X_test.shape)


X_train:

[[  0]
 [  0]
 [  6]
 [ 10]
 [ -2]
 [-10]
 [  8]
 [  3]
 [  1]
 [  0]
 [ -8]
 [  1]
 [ -2]
 [  4]
 [  6]
 [-12]]
(814, 269, 16, 1)

X_test:

[[  0]
 [  0]
 [ -4]
 [ -1]
 [ -2]
 [ 11]
 [  8]
 [ -6]
 [ 11]
 [  5]
 [ 10]
 [  5]
 [  2]
 [  7]
 [  1]
 [-20]]
(350, 269, 16, 1)


In [48]:
y_test

array([2, 3, 2, 3, 3, 6, 4, 3, 5, 3, 6, 3, 0, 0, 3, 1, 6, 4, 5, 3, 2, 6,
       1, 0, 2, 3, 1, 6, 4, 3, 5, 6, 6, 6, 3, 5, 2, 2, 3, 4, 0, 1, 4, 3,
       4, 2, 2, 6, 4, 5, 2, 1, 6, 1, 0, 5, 5, 6, 6, 6, 1, 0, 4, 4, 2, 1,
       1, 1, 3, 2, 3, 3, 1, 1, 0, 3, 2, 2, 6, 4, 3, 1, 3, 2, 2, 6, 6, 4,
       4, 5, 2, 0, 5, 2, 3, 1, 1, 0, 6, 0, 4, 3, 5, 4, 6, 2, 1, 1, 6, 3,
       0, 2, 6, 4, 1, 1, 0, 0, 6, 3, 3, 2, 0, 5, 3, 0, 5, 5, 5, 6, 6, 4,
       5, 3, 5, 1, 5, 6, 1, 5, 5, 6, 2, 6, 6, 0, 2, 3, 2, 3, 0, 3, 5, 4,
       2, 2, 5, 1, 5, 2, 6, 2, 3, 5, 3, 2, 1, 2, 5, 4, 3, 1, 4, 4, 5, 5,
       3, 1, 4, 3, 0, 3, 2, 4, 2, 6, 4, 6, 1, 5, 0, 4, 3, 6, 4, 4, 5, 5,
       4, 3, 1, 6, 4, 5, 2, 1, 4, 6, 3, 0, 2, 2, 5, 5, 5, 6, 2, 3, 3, 4,
       3, 6, 5, 1, 1, 4, 3, 5, 2, 3, 0, 3, 3, 4, 2, 3, 1, 0, 2, 2, 2, 2,
       0, 1, 2, 5, 5, 0, 0, 2, 4, 5, 5, 3, 6, 1, 4, 1, 6, 4, 2, 5, 4, 5,
       6, 0, 3, 4, 3, 4, 1, 2, 6, 2, 3, 5, 2, 1, 4, 3, 1, 3, 2, 0, 4, 6,
       1, 1, 6, 3, 4, 3, 3, 4, 6, 5, 4, 1, 4, 0, 3,

## Building and Compiling Model

In [75]:
# Construct model 1
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))

    model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))

    model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(GlobalAveragePooling2D())

    model.add(Dense(5, activation='softmax'))

In [77]:
# top-k category accuracy
import functools
import keras
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',top3_acc])

In [78]:
# Define bath and epochs
batch_size = 256
epochs = 3
checkpoint_file = '../models/model_checkpoints/original_window_2.h5'

lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=20, min_lr=0.0000001)
mcp_save = ModelCheckpoint(checkpoint_file, save_best_only=True, monitor='val_loss', mode='min')
cnnhistory=model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test), callbacks=[mcp_save, lr_reduce])

Train on 814 samples, validate on 350 samples
Epoch 1/3
814/814 [==============================] - 4s 4ms/step - loss: 4.3884 - accuracy: 0.1572 - top3_acc: 0.4472 - val_loss: 1.5433 - val_accuracy: 0.4114 - val_top3_acc: 0.7143
Epoch 2/3
814/814 [==============================] - 3s 4ms/step - loss: 2.2310 - accuracy: 0.4300 - top3_acc: 0.7101 - val_loss: 1.9975 - val_accuracy: 0.3086 - val_top3_acc: 0.7114
Epoch 3/3
814/814 [==============================] - 3s 4ms/step - loss: 2.2580 - accuracy: 0.3968 - top3_acc: 0.7260 - val_loss: 1.6409 - val_accuracy: 0.3657 - val_top3_acc: 0.7086


In [ ]:
cnnhistory.history['accuracy'][-2]

In [ ]:
cnnhistory.history.keys()

In [ ]:
cnnhistory.history['accuracy'].index(np.sort(cnnhistory.history['accuracy'])[-1])

In [ ]:
import matplotlib.pyplot as plt

# Model Summary

model.summary()

# PLT History info
plt.plot(cnnhistory.history['loss'])
plt.plot(cnnhistory.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()